## Conctere Model
- $min \ 2x_1 + 3x_2$
- S.T.
    - $3x_1 + 4x_2 \geq 1$
    - $x_1, x_2 \geq 0$

In [1]:
import pyomo.environ as pyo

model = pyo.ConcreteModel()
model.x = pyo.Var([1, 2], domain=pyo.NonNegativeReals)
model.OBJ = pyo.Objective(expr=2*model.x[1] + 3*model.x[2])
model.Constraint1 = pyo.Constraint(expr=3*model.x[1] + 4*model.x[2] >= 1)

In [2]:
model.pprint()

1 Var Declarations
    x : Size=2, Index={1, 2}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 2*x[1] + 3*x[2]

1 Constraint Declarations
    Constraint1 : Size=1, Index=None, Active=True
        Key  : Lower : Body            : Upper : Active
        None :   1.0 : 3*x[1] + 4*x[2] :  +Inf :   True

3 Declarations: x OBJ Constraint1


## Abstract Model
- $min \ \sum_{j=1}^n \ c_jx_j$
- S.T.
    - $\sum_{j=1}^n a_{ij}x_j \geq b_i$ for $i = 1...m$
    - $x_j \geq 0$ for $j = 1...n$

In [ ]:
import pyomo.environ as pyo

model = pyo.AbstractModel()

model.m = pyo.Param(within=pyo.NonNegativeIntegers)
model.n = pyo.Param(within=pyo.NonNegativeIntegers)

model.j = pyo.RangeSet(1, model.n)
model.i = pyo.RangeSet(1, model.m)

model.c = pyo.Param(model.j)
model.a = pyo.Param(model.i, model.j)
model.b = pyo.Param(model.i)

model.x = pyo.Var(model.j, domain=pyo.NonNegativeReals)

def obj_expression(m):
    return pyo.summation(m.c, m.x)

model.OBJ = pyo.Objective(rule=obj_expression)

def ax_constraint_rule(m, i):
    return sum(m.a[i,j] * m.x[j] for j in m.j) >= m.b[i]

model.AxbConstraint = pyo.Constraint(model.i, rule=ax_constraint_rule)


In [ ]:
# Create data dictionary
data = {
    None: {
        'm': {None: 2},  # 2 constraints
        'n': {None: 2},  # 2 variables
        'c': {1: 2, 2: 3},  # objective coefficients
        'a': {(1, 1): 3, (1, 2): 4, (2, 1): 1, (2, 2): 1},  # constraint coefficients
        'b': {1: 1, 2: 1}  # RHS of constraints
    }
}

instance = model.create_instance(data)
instance.pprint()

2 RangeSet Declarations
    i : Dimen=1, Size=2, Bounds=(1, 2)
        Key  : Finite : Members
        None :   True :   [1:2]
    j : Dimen=1, Size=2, Bounds=(1, 2)
        Key  : Finite : Members
        None :   True :   [1:2]

5 Param Declarations
    a : Size=4, Index=i*j, Domain=Any, Default=None, Mutable=False
        Key    : Value
        (1, 1) :     3
        (1, 2) :     4
        (2, 1) :     1
        (2, 2) :     1
    b : Size=2, Index=i, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :     1
          2 :     1
    c : Size=2, Index=j, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :     2
          2 :     3
    m : Size=1, Index=None, Domain=NonNegativeIntegers, Default=None, Mutable=False
        Key  : Value
        None :     2
    n : Size=1, Index=None, Domain=NonNegativeIntegers, Default=None, Mutable=False
        Key  : Value
        None :     2

1 Var Declarations
    x : Size=2, Index=j
        Key : Lower 